In [25]:
# !pip install kaggle
# !cd ~/.kaggle
# !mv ~/Downloads/kaggle.json ./
# !chmod 600 ./kaggle.json
!pip install category_encoders


     |████████████████████████████████| 82 kB 1.2 MB/s             
     |████████████████████████████████| 9.6 MB 9.0 MB/s            
     |████████████████████████████████| 233 kB 8.3 MB/s            
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
!kaggle competitions download -c caltech-cs155-2022-mp1

 94%|███████████████████████████████████▊  | 12.0M/12.7M [00:00<00:00, 18.1MB/s]
100%|██████████████████████████████████████| 12.7M/12.7M [00:00<00:00, 16.9MB/s]


In [26]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import category_encoders as ce


Import and Parse Data

In [33]:
train_data = pd.read_csv('data/LOANS_TRAIN.csv')
test_data = pd.read_csv('data/LOANS_TEST.csv')

#Remove id column
train_data = train_data.drop(columns=['id'])

#Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)

ce_OHE = ce.OneHotEncoder(cols=['loan_status', 'application_type', 'initial_list_status'])
train_data = ce_OHE.fit_transform(train_data)

#Replace grade with numerical value
grades = {'A':6, 'B':5,'C':4, 'D':3, 'E':2, 'F':1}
train_data['grade'].replace(grades, inplace=True)

#Replace subgrade with numerical value
subgrades = {
    'A1': 6.8, 'A2': 6.6, 'A3':6.4, 'A4':6.2, 'A5':6.0,
    'B1': 5.8, 'B2': 5.6, 'B3':5.4, 'B4':5.2, 'B5':5.0,
    'C1': 4.8, 'C2': 4.6, 'C3':4.4, 'C4':4.2, 'C5':4.0,
    'D1': 3.8, 'D2': 3.6, 'D3':3.4, 'D4':3.2, 'D5':3.0,
    'E1': 2.8, 'E2': 2.6, 'E3':2.4, 'E4':2.2, 'E5':2.0,
    'F1': 1.8, 'F2': 1.6, 'F3':1.4, 'F4':1.2, 'F5':1.0
}
train_data['sub_grade'].replace(subgrades, inplace=True)

#Replace home_ownership with numerical values
ownership = {'RENT': -3, 'MORTGAGE': -1, 'OWN': 1}
train_data['home_ownership'].replace(ownership, inplace=True)

print(train_data[:2])

   loan_amnt  term_(months)  int_rate  installment grade sub_grade emp_title  \
0       5000             36     10.65       162.87     5       5.6       NaN   
1       2500             60     15.27        59.83     4       4.2     Ryder   

  emp_length home_ownership  annual_inc  ... revol_bal revol_util total_acc  \
0  10+ years             -3     24000.0  ...     13648       83.7         9   
1   < 1 year             -3     30000.0  ...      1687        9.4         4   

  initial_list_status_1 initial_list_status_2 application_type_1  mort_acc  \
0                     1                     0                  1       NaN   
1                     1                     0                  1       NaN   

  pub_rec_bankruptcies  loan_status_1  loan_status_2  
0                  0.0              1              0  
1                  0.0              0              1  

[2 rows x 30 columns]


In [ ]:
print(train_data.values[0])
print(train_data.values[0][5])
count = 0

correct = 0
total = 0
for datapoint in train_data.values:
    credit_score = datapoint[5]
    outcome = datapoint[-1]
    if outcome == "Fully Paid":
        ground_truth = 1
    else:
        ground_truth = 0
    if credit_score == "A" or credit_score == "B":
        prediction = 1
    else:
        prediction = 0
    if ground_truth == prediction:
        correct += 1
    total += 1
print(f"ACCURACY: {correct/total}")


